# 4_Data Preparation and Exploration
## Porto Seguro’s Safe Driver Prediction
# Introduction

노트북은 다음과 같은 주요 섹션으로 구성됩니다.

1. Visual inspection of your data
2. Defining the metadata
3. Descriptive statistics
4. Handling imbalanced classes
5. Data quality checks
6. Exploratory data visualization
7. Feature engineering
8. Feature selection
9. Feature scaling

# Loading packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer 

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)


### Loading data

In [ ]:
train = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/test.csv')

### Data at first sight

데이터에 대한 정보들(밑의 head와 tail에서 볼 수 있음):

- 비슷한 그룹인 Feature들은 비슷한 이름을 가지고 있다 (예를 들어, ind, reg, car, calc).
- **bin** 을 가진 Feature는 **Binary feature**임을 나타내고, **cat** 를 가진 Feature는 **Categorical feature**임을 나타낸다.
- 이외의 Feature들은 **Continious** 혹은 **Ordinal feature** 이다.
- 값이 -1 인 관측치는 **결측값(NaN)**을 의미한다.
- **Target** 컬럼은 Policy holder(보험 계약자=보험료를 납입할 의무를 가진 사람)에게 청구 적용 여부(Y/N)를 나타낸다.

### train 데이터의 전체적인 모습을 확인하기 위하여 앞부분(head)과 뒷부분(tail)을 먼저 확인해보자.

In [ ]:
train.head()

In [ ]:
train.tail()

- 추후에 14개의 **Categorial variables**를 더미화 시킬 것이다.
- ex) ps_ind_02_cat(04, 05), ps_car_01_cat(01~11) : 총14개
- 접미사로 **bin**이 붙은 **variables**는 이미 0과 1로 구성되어 있으므로 따로 더미화시킬 필요가 없다.(Binary feature)

### 밑에서는 train 데이터의 행과 열 수를 살펴보자.

In [ ]:
train.shape


59개의 변수와 595.212개의 행이 있다. 테스트 데이터에 같은 수의 변수가 있는지 보자.
훈련 데이터에 중복 행이 있는지도 살펴보자.

In [ ]:
train.drop_duplicates()
train.shape

중복 행이 없다.

In [ ]:
test.shape

테스트 세트에서 하나의 변수(컬럼)가 누락되었지만 이것이 타겟 변수라서 괜찮다.
이제 각 유형의 변수가 몇 개인지 조사해 보자.

따라서 나중에 14개의 범주형 변수에 대한 더미 변수를 만들 수 있다. bin 변수는 이미 바이너리이며 더미화가 필요하지 않다.(바이너리: 0,1로 나타나져있어서 더미화로 나눌 필요 없음)

In [ ]:
train.info()

- 데이터들의 타입이 정수형이거나 소수형이다.
- 결측값이 -1로 되어 있어 결측값이 없는 것으로 확인되는데 이것은 후에 다룰 것.

### Metadata

:[데이터에 관한 구조화된 데이터로, 다른 데이터를 설명해 주는 데이터]

데이터 관리를 용이하게 하기 위해 variables의 정보들을 DataFrame으로 저장해보자. 

이렇게 저장한 데이터프레임은, 분석에 필요한 특정한 variables를 선택할 때, 시각화를 할 때, 모델링을 할 때 등에 도움이 될 것이다.

구체적으로 저장해야 할 것들은 다음과 같다.

- role: input, ID, target
- level: nominal, interval, ordinal, binary
- keep: True or False
- dtype: int, float, str

In [ ]:
data = []
for f in train.columns:
    #Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
        
    #Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
        
    #Defining the data type
    dtype = train[f].dtype
    
    #Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname' : f,
        'role': role,
        'level': level,
        'keep' : keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns = ['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace = True)

In [ ]:
meta

### 예시로 level이 nominal인 데이터의 인덱스를 추출해보자.

In [ ]:
meta[(meta.level == 'nominal') & (meta.keep)].index

### role과 level에 따른 target의 수를 아래를 통해 확인해보자.

In [ ]:
meta[(meta.level == 'nominal')].index

In [ ]:
pd.DataFrame({'count': meta.groupby(['role', 'level'])['role'].size()}).reset_index()

## ****Descriptive statistics****

: 기술 통계(확보한 데이터를 이해하기 쉬운 수치로 요약하는 기법)

- 데이터프레임에 describe 메소드를 사용하여 기술통계량을 살펴보도록 하자.
- 그러나 describe 메소드는 categorical, id variable의 기술통계량은 계산해주지 않는다.  그러니 추후에 categorical variables를 살펴보도록 하자.

메타 데이터를 이용하여 손쉽게 기술통계량을 계산할 수 있다.

### ****Interval variables****

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

### reg variables
![Untitled](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/9f8d4b3f-a8e4-48e8-b87f-4b675f7735ff/Untitled.png)

- ps_car_12 및 ps_car_14에 누락된 값이 있다.(min = -1)
- 변수에 따라 범위가 다르기 때문에 이 variables 역시, 스케일링이 필요해보인다.



### car variables
![Untitled](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/3271c04c-db48-4f91-8cee-bf16d895cee4/Untitled.png)
* ps_car_12 및 ps_car_15에 누락된 값이 있다.
* 다시, 범위가 다르며 스케일링을 적용할 수 있다.



### calc variables
![Untitled](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/66fdd8f5-55a3-45f7-9400-bf969d3b6a92/Untitled.png)
- 결측치가 없다.(min = 0)
- 이 variables는 최대값이 0.9인것으로 보아 일종의 비율인 것 같다.
- 모든 3개의 *_calc* variables은 매우 비슷한 분포를 가지고 있다. & 범위도 같음

**전반적으로**, interval variables들 간의 범위가 상대적으로 좁음을 확인 가능하다. (제일 큰 게 -1~4.0)

아마도 데이터를 익명화시키기 위하여 몇몇 변환 작업(예를 들어 log)가 이미 적용된 것은 아닐까 생각된다.

### 기술통계를 살펴봄으로써 다음과 같은 결과를 얻었다

- Feature 내부에 결측치의 존재 유무(-1의 유무)
- min과 max를 비교함으로써 스케일링의 필요성 판단(범위 크기 차이)
- max 값을 살펴봄으로써 변수의 값이 비율인지 판단(calc)

### Ordinal variables

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

* 하나의 누락된 변수: ps_car_11
* 다양한 범위를 처리하기 위해 스케일링을 적용할 수 있습니다.

### Binary variables

In [ ]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

train 데이터의 target 평균은 0.036448로 3.645% 인데, 결과값이 매우 불균형함을 알 수 있다.
![Untitled](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/20ea0200-888a-453f-b10a-eafb85a8dce9/Untitled.png)
평균값을 통해 대다수의 variables가 0이라고 결론내릴 수 있다.


## Handling imbalanced classes

위에서 언급했듯이 target=1인 레코드의 비율은 target=0보다 훨씬 적다. 이것은 정확도가 높지만 실제로는 부가 가치가 있는 모델로 이어질 수 있다. 이 문제를 처리하기 위한 두 가지 가능한 전략은 다음과 같다.

* target=1인 오버샘플링 레코드
* target=0인 언더샘플링 레코드

물론 더 많은 전략이 있으며 MachineLearningMastery.com은 좋은 개요를 제공한다. 우리는 훈련 세트가 상당히 크기 때문에 언더샘플링을 할 수 있다.

In [ ]:
desired_apriori = 0.10

#get the indices per target value
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

#get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

#Calculate the undersampling rate and resulting number of records with target = 0
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target = 0: {}'.format(undersampling_rate))
print('Number of records with target = 0 after undersampling: {}'.format(undersampled_nb_0))

#Randomly select records with target = 0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state = 37, n_samples = undersampled_nb_0)

#Construct list with remaining indeices
idx_list = list(undersampled_idx) + list(idx_1)

#Return undersample data frame
train = train.loc[idx_list].reset_index(drop = True)

# Data Quality Checks
## Checking missing values

누락은 -1로 표시된다.

In [ ]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

* ps_car_03_cat 및 ps_car_05_cat에는 결측값이 있는 레코드의 비율이 높다. 이러한 변수를 제거하자.
* 결측값이 있는 다른 범주형 변수의 경우 결측값 -1을 그대로 둘 수 있다.
* ps_reg_03(연속)에 모든 레코드의 18%에 대한 결측값이 있다. 평균으로 교체하자.
* ps_car_11(서수)에는 값이 누락된 레코드가 5개뿐이다. 모드로 교체하자.
* ps_car_12(연속)에는 누락된 값이 있는 레코드가 1개뿐이다. 평균으로 교체하자.
* ps_car_14(연속)에 모든 레코드의 7%에 대한 결측값이 있다. 평균으로 교체하자.

In [ ]:
#Imputer =SimpleImputer(missing_values=np.nan, strategy='mean')#Imputing with the mean or mode

In [ ]:
# Dropping the variables with too many missing values
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace = True, axis =1)
meta.loc[(vars_to_drop), 'keep'] = False #Updating the meta

In [ ]:
mean_imp = SimpleImputer(missing_values = -1, strategy = 'mean')
mode_imp = SimpleImputer(missing_values = -1, strategy = 'most_frequent')
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

## Checking the cardinality of the categorical variables

카디날리티는 전체 행에 대한 특정 컬럼의 중복 수치를 나타내는 지표입니다.
중복도가 높으면 카디날리티가 낮으며, 중복도가 낮으면 카디날리티가 높습니다.
카디날리티는 상대적인 개념으로 이해해야 합니다.

따라서 카디날리티는 variable 내에서 다른 value의 개수를 말합니다. 우리는 추후 categorical variables를 더미화시킬 것인데, variables 내에 다른 value들이 얼마나 많은지 체크해봐야 합니다.Value들이 많을 경우, 수 많은 더미 변수들이 만들어질 수 있기 때문입니다.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Variabel {} has {} distinct values'.format(f, dist_values))

합리적이긴 하지만, ps_car_11_cat는 104개로 매우 많은 Value를 가지고 있습니다.

EDIT : 최초 작성자분은 104개의 Value에 대해 가공을 하여 데이터 손실이 있었던 것으로 보입니다. 이후 최초 작성자분은 Oliver의 커널을 활용한 방법을 사용했습니다.

In [ ]:
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series * ( 1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series = None,
                 tst_series = None,
                 target = None,
                 min_samples_leaf = 1,
                 smoothing = 1,
                 noise_level = 0):
    '''
    Smoothing is computed like in the following paper by Dniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    '''
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis =1)
    # Conpute target mean
    averages = temp.groupby(by = trn_series.name)[target.name].agg(['mean', 'count'])
    #compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
    #Apply average function to all target data
    prior = target.mean()
    #The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1-smoothing) + averages['mean'] * smoothing
    averages.drop(['mean', 'count'], axis = 1, inplace = True)
    #Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
    averages.reset_index().rename(columns = {'index' : target.name, target.name: 'average'}),
    on = trn_series.name,
    how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    #pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns = {'index' : target.name, target.name: 'average'}),
        on = tst_series.name,
        how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    #pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
train_encoded, test_encoded = target_encode(train['ps_car_11_cat'],
                                           test['ps_car_11_cat'],
                                           target = train.target,
                                           min_samples_leaf = 100,
                                           smoothing = 10,
                                           noise_level = 0.01)

train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis = 1, inplace=True)
meta.loc['ps_car_11_cat','keep'] = False  #Updating the meta
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis = 1, inplace=True)

# Exploratory Data Visualization 데이터 시각화

target 값이 1인 categorical variables와 customers의 비율을 살펴보도록 합시다.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize = (20, 10))
    #Calculate the percentage of target = 1 per category value
    cat_perc = train[[f, 'target']].groupby([f], as_index = False).mean()
    cat_perc.sort_values(by='target', ascending = False, inplace = True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax= ax, x = f, y= 'target', data =cat_perc, order = cat_perc[f])
    plt.xlabel(f, fontsize =18)
    plt.ylabel('% target', fontsize = 18)
    plt.tick_params(axis = 'both', which = 'major', labelsize = 18)
    plt.show();

막대 그래프들을 통해 결측값이 있는 variables들을 확인할 수 있습니다. 앞서 결측값들을 치환했는데, categorical variables들은 따로 치환을 하지 않았습니다. 최빈값으로 대체하는 것보다 분리된 category value로서 결측값을 보는 것이 더 좋은 방법일 수 있습니다.

결측값을 가지고 있는 Customer들이 다른 Value들에 비하여 훨씬 높은 target 평균을 가지고 있기 때문입니다!

## Interval variables

interval variables의 상관관계를 확인하고자 합니다. Heatmap은 Variables 간의 상관관계를 확인하는데 매우 효율적입니다. 하기 코드는 an example by Michael Waskom에 기반하고 있습니다.

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()
    
    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap = True)
    
    fig, ax = plt.subplots(figsize = (10, 10))
    sns.heatmap(correlations, cmap=cmap, vmax = 1.0, center = 0, fmt = '.2f',
               square = True, linewidths = .5, annot = True, cbar_kws = {'shrink': .75})
    plt.show();
    
v = meta[(meta.level == 'interval') & (meta.keep)].index
corr_heatmap(v)

강한 상관관계를 가지고 있는 Variables들은 다음과 같습니다. :

* ps_reg_02 and ps_reg_03 (0.7)
* ps_car_12 and ps_car13 (0.67)
* ps_car_12 and ps_car14 (0.58)
* ps_car_13 and ps_car15 (0.67)

Seaborn의 pair plot을 사용하면 variables들의 (선형) 관계를 손쉽게 시각화할 수 있습니다. 하지만 히트맵이 상관관계가 있는 variables들의 관계들을 시각화해주고 있기 때문에, 우리는 높은 상관관계를 보이는 variables들을 분리해서 보고자 합니다.

Note 프로세스의 속도를 높이기 위하여 train 데이터의 sample을 사용합니다.

In [ ]:
s = train.sample(frac = 0.1)


train 데이터에서 10%의 데이터를 샘플링합니다.

* train.shape ▶ (216940, 57)
* s.shape ▶ (21694, 57)

### ps_reg_02 and ps_reg_03 (0.7)
회귀선이 보여주듯이, 두 variables들 간에는 선형 상관관계를 살펴볼 수 있습니다. hue 파라미터를 통해 target=0과 target=1에 대한 회귀선이 동일함을 알 수 있습니다.

In [ ]:
sns.lmplot(x = 'ps_reg_02', y='ps_reg_03', data=s, hue = 'target', palette = 'Set1', scatter_kws = {'alpha':0.3})
plt.show()

### ps_car_12 and ps_car_13 (0.67)

In [ ]:
sns.lmplot(x = 'ps_car_12', y = 'ps_car_13', data =s, hue = 'target', palette = 'Set1', scatter_kws = {'alpha': 0.3})
plt.show()

### ps_car_12 and ps_car_14 (0.58)

In [ ]:
sns.lmplot(x = 'ps_car_12', y = 'ps_car_14', data = s, hue='target', palette = 'Set1', scatter_kws = {'alpha':0.3})
plt.show()

### ps_car_13 and ps_car_15 (0.67)

In [ ]:
sns.lmplot(x = 'ps_car_15', y = 'ps_car_13', data = s, hue = 'target', palette = 'Set1', scatter_kws = {'alpha':0.3})
plt.show()

이제는 어떤 correlated variables를 유지할지 결정해야 합니다. 이를 위하여 우리는 Principal Component Analysis (PCA), 주성분 분석을 실시하여 variables의 dimensions를 줄일 수 있습니다. 하지만 correlated variables의 수가 적은만큼, 우리는 모델이 heavy-lifting을 하도록 해야합니다.

## Checking the correlations between ordinal variables

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
corr_heatmap(v)

ordinal variables는 큰 상관관계를 가지고 있지 않은 것으로 보입니다. 반면에 target 값으로 그룹화할 때 분포가 어떻게 될지 확인할 수 있습니다.

# Feature engineering

## Creating dummy variables

categorical variables는 어떤 순서나 경중이 담겨있지 않습니다. 예를 들어서 카테고리 2는 카테고리 1보다 2배의 값을 가지고 있지 않습니다. 이 문제는 더미 데이터를 만들어줌으로써 해결할 수 있습니다. 첫 번째 dummy variable의 정보는 원래 variables의 범주에 대해 생성된 다른 dummy variable에서 파생될 수 있으므로 삭제해주도록 합니다.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('before dummification we have {} variables in train'.format(train.shape[1]))
train = pd.get_dummies(train, columns = v, drop_first = True)
print('After dummification we have {} variables in train'.format(train.shape[1]))

dummy variables는 training 데이터 세트에 52개의 variables를 추가했습니다.

## Creating interaction variables

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree = 2, interaction_only = False, include_bias = False)
interactions = pd.DataFrame(data = poly.fit_transform(train[v]), columns = poly.get_feature_names(v))
interactions.drop(v, axis = 1, inplace = True) #poly 처리가 되지 않은 기존의 열들을 삭제한다.

#interactions와 train 데이터를 합쳐준다
print('Before crating interactions we have {} variabbles in train'.format(train.shape[1]))
train = pd.concat([train, interactions], axis = 1)
print('After creating interactions we have {} variables in train'.format(train.shape[1]))

PolynomialFeatures는 다항차수 변환을 진행을 도와주는 클래스입니다. 상기 코드의 경우 degree를 2로 설정했으니 2차항 변수로 만들어주는 것입니다.

이를 통해 train 데이터에 interaction variables를 추가할 수 있습니다. get_feature_names 메소드 덕분에 열 이름을 할당할 수 있습니다.

# Feature selection

## Removing features with low or zero variance

개인적으로 작성자는 분류기의 알고리즘이 유지할 features를 선택하는 것을 선호한다고 합니다. 하지만 우리 스스로 할 수 있는 일도 있습니다. 분산이 0이거나 아주 적은 features들을 제거하는 것입니다.


이를 위해 사이킷런의 VarianceThreshold라는 메소드를 사용할 수 있습니다. 기본적으로 이 메소드는 분산 값이 0인 features들을 제거해줍니다.


하지만 저희는 이전 단계에서 이미 분산이 0인 features가 없음을 확인했기 때문에, 우리는 1% 미만의 분산이 있는 features들을 제거해주고자 합니다. 이를 통해 우리는 31개의 variables를 제거하게 됩니다.

In [ ]:
selector = VarianceThreshold(threshold = .01)
selector.fit(train.drop(['id', 'target'], axis = 1)) # Fit to train without id and target variables

f = np.vectorize(lambda x: not x) #Function to toggle boolean array elements

v = train.drop(['id', 'target'], axis = 1).columns[f(selector.get_support())]
print('{} variables have too low variance.'.format(len(v)))
print('these variables are {}'.format(list(v)))

만약 우리가 분산에 기반하여 선택을 진행한다면 많은 variables들을 잃게 될 것입니다. 하지만 우리는 많은 variables를 가지고 있지 않기 때문에, 분류기가 직접 선택하도록 합니다. variables가 더 많은 데이터 셋이라면 처리 시간을 줄여줄 수 있을 것입니다.

사이킷런은 [feature selecetion methods]를 제공합니다. 이 메소드 중 하나가 'SelectFromModel' 인데, 다른 분류기에서 최상의 feature를 선택하고 기능을 계속할 수 있도록 합니다. 아래를 통해 랜덤 포레스트를 어떻게 사용하는지 확인해보도록 합니다.

# Selecting features with a Random Forest and SelectFromModel

우리는 랜덤 포레스트의 feature importances에 따라 feature 선택의 기준을 삼습니다. SelectFromModel을 통하여 유지할 variables의 숫자를 구체화할 수 있습니다. feature의 중요도에 대한 임곗값을 수동으로 설정할수 있지만, 우리는 단순히 50% 이상의 최적의 variables를 선택해보도록 합시다.

>하기의 코드는 이곳에서 가져왔습니다. GitHub repo of Sebastian Raschka.(https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch04/ch04.ipynb)



In [ ]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_

indices = np.argsort(rf.feature_importances_)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))

SelectFromModel을 사용하면 사용할 사전 적합 분류기와 기능 중요도에 대한 임계값을 지정할 수 있습니다. get_support 메소드를 사용하여 기차 데이터의 변수 수를 제한할 수 있습니다.

In [ ]:
sfm = SelectFromModel(rf, threshold = 'median', prefit = True)
print('Number of features before selection: {}'.format(X_train.shape[1]))
n_features = sfm.transform(X_train).shape[1]
print('Number of features after selection: {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])

In [ ]:
train = train[selected_vars + ['target']]

# Feature scaling
이전에 언급했듯이, 우리는 train 데이터에 정규화를 진행할 수 있습니다. 몇몇 분류기에서는 더 나은 결과를 가져올 수 있을 것입니다.

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(train.drop(['target'], axis = 1))

우왕 끝!